### Necessary packages

In [ ]:
# Common imports
import glob
import random
import os
import re
import absl.logging

absl.logging.set_verbosity(absl.logging.ERROR)# Avoid print error

from datetime import datetime
import numpy as np 
from numpy.random import seed
import pandas as pd
pd.options.mode.chained_assignment = None  # default='warn'
pd.options.mode.chained_assignment = None # avoid error 
pd.options.display.float_format = '{:.5f}'.format # No scientific annotation when print dataframe
from functools import reduce
  
# ADA and Modeling imports
import math
import sklearn
from sklearn.preprocessing import StandardScaler, RobustScaler,MinMaxScaler, LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split,KFold,GridSearchCV,RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error, median_absolute_error, mean_squared_log_error, accuracy_score
from IPython.display import clear_output
import statistics
from scipy.stats import stats
import tensorflow as tf
from tensorflow import keras
#from tsmoothie.smoother import *

# To plot pretty figures
import seaborn as sns
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Build function
#from HelperFunctions import *
#from Implementation import *

# To generate an stable output across runs
rnd_seed = 42
rnd_gen = np.random.default_rng(rnd_seed)

# MAchine learing packages 
import tensorflow 
from keras.layers import SimpleRNN
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import metrics 
from tensorflow.keras.metrics import RootMeanSquaredError
from tensorflow.keras.optimizers import RMSprop, Adam, SGD, Adamax, Nadam
from tensorflow.keras.activations import selu,elu,relu,tanh
from tensorflow.keras.initializers import RandomNormal
from bayes_opt import BayesianOptimization
from tensorflow.keras.callbacks import EarlyStopping
import time
from skopt import gp_minimize
from skopt.space import Real, Categorical, Integer
from skopt.utils import use_named_args
import tensorflow_probability as tfp
from skopt import gp_minimize
from skopt.space import Real, Categorical, Integer
from skopt.utils import use_named_args
from keras import backend as K
from keras import Input #
from keras.models import Sequential, load_model
import keras.backend as K 
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.layers import Dense, Dropout, LSTM, GRU, InputLayer, Flatten, Conv1D, Activation, LeakyReLU
LeakyReLU = LeakyReLU(alpha=0.1)

n_cpu = os.cpu_count()
print("Number of CPUs in the system:", n_cpu)

### Data

In [ ]:
''' This code required the right version of packages
     like numpy and tensor '''

In [ ]:
'''Data normlization and reshaping'''
df = pd.read_csv('d_f_RNN.csv')
scaler = MinMaxScaler(feature_range=(0, 1))
X_scaled = scaler.fit_transform(df.iloc[:,:-1])
X_df =pd.DataFrame(X_scaled)

y_scaled = scaler.fit_transform((df.iloc[:,-1].to_numpy()).reshape(-1,1))#.reshape(-1,1)
y_df =pd.DataFrame(y_scaled)

# Selecting features and label for training 

features = X_df.astype('float32') # Training data
features[2] = features[2].astype(int) # Strat as integer
label = y_df.astype('float32')#.to_numpy().reshape(-1,1) # 'Labels'
# Reshaping data into n_samples x timestep to be feed into the GRU netwok
def Reshape_df(X, y, time_steps):
    Xs, ys = [], []
    for i in range(len(X) - time_steps):
        v = X.iloc[i:(i + time_steps)].values
        Xs.append(v)
        ys.append(y.iloc[i + time_steps])
    return np.array(Xs), np.array(ys)

X, y = Reshape_df(features, label, 100)

print(X.shape,y.shape)


'''Data Splitting'''

s = 0.8309
split = int((s)*X.shape[0])
X_train = X[:split,:,:]
y_train = y[:split,:]
X_test =  X[split:,:,:]
y_test =  y[split:,:]

sets = [X, X_test, y, y_test]
sets_n = ['X_train', 'X_test', 'y_train', 'y_test']

for i,im in enumerate(sets):
    print(f'{sets_n[i]}:',im.shape)

In [ ]:
from skopt.callbacks import DeltaYStopper
search_space = [Categorical([12,34,60,120], name='batch_size'),
                Categorical([10,30], name='epochs'),
                Real(1e-8, 1e-3,'log-uniform',name='learning_rate'),
                Categorical(['relu', 'tanh','selu','elu'], name='activation'),
                Real(0.0, 0.5, name='dropout_rate'),
                Categorical(['Adam', 'SGD','rmsprop'], name='optimizer'),
                Integer(10, 200, name='num_units'),
                Integer(1, 3, name='num_layers'),
                Real(0.1,0.9,name='lambda1'), 
                Real(0.1,0.9,name='lambda2')]

# Define the objective function to be minimized
@use_named_args(search_space)
def Hybrid_model(**params):


    batch_size = params['batch_size']
    epochs = params['epochs']
    learning_rate = params['learning_rate']
    activation = params['activation']
    dropout_rate = params['dropout_rate']
    optimizer = params['optimizer']
    num_units = params['num_units']
    num_layers = params['num_layers']
    l1 = params['lambda1']
    l2 = params['lambda2']

    if optimizer == 'Adam':
        opt = keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'SGD':
        opt = keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = keras.optimizers.RMSprop(learning_rate=learning_rate)



        
    # Build the model
    Hybrid = Sequential()
    Hybrid.add(Input(shape=(X_train_kfold.shape[1],X_train_kfold.shape[2]), name='input_layer'))
                     
    for i in range(num_layers):
        if i == num_layers - 1:
            Hybrid.add(GRU(units=num_units,
                           activation=activation,
                           dropout=dropout_rate,
                           return_sequences=False,
                           name=f'Hidden_layer_{i+1}'))
        else:
            # Not the last GRU layer
            Hybrid.add(GRU(units=num_units,
                           activation=activation,
                           dropout=dropout_rate,
                           return_sequences=True,
                           name=f'Hidden_layer_{i+1}'))
    Hybrid.add(Dense(1))
    Hybrid.compile(loss=rmse, optimizer=opt, metrics=[r_squared])

    # Train the model on the training data for this fold
    
    # Return the validation loss as the objective value to be minimized

    # Evaluate the model on the validation data for this fold
    Hybrid.fit(X_train_kfold,y_train_kfold,
                   batch_size=batch_size,
                   epochs=epochs,
                   verbose=1)
    val_loss, val_r_squared = Hybrid.evaluate(X_val_kfold, y_val_kfold, verbose=1)
    return val_loss


In [ ]:
# Define per-fold score containers
from sklearn.model_selection import KFold
import pickle
from skopt.callbacks import DeltaYStopper
r2_per_fold = []
loss_per_fold = []
rmse_per_fold = []
gru_best_score = []

#l1 = 0.5
#l2 = 0.5
kfold = KFold(n_splits=6)
fold_no =1
parameters_fold = [None]*6
#for train_index, val_index in kfold.split(X_train, y_train):
for i, (train_index, val_index) in enumerate(kfold.split(X_train, y_train)):       
    print(i,fold_no)
    X_train_kfold, X_val_kfold = X_train[train_index], X_train[val_index]
    y_train_kfold, y_val_kfold = y_train[train_index], y_train[val_index]
        # Best model for th k-fold
    loss = True
    delta_stopper = DeltaYStopper(delta=0.1)
    result = gp_minimize(func=Hybrid_model,
                     dimensions=search_space,
                     n_calls=10,
                    random_state=42,
                    callback=[delta_stopper],
                    verbose=True)
    clear_output(wait = True)
    print('Saving',fold_no)
    params0 = {param.name: value for param, value in zip(search_space, result.x)}
    parameters_fold[i] = {'fold':fold_no,'parameters':params0}
    clear_output(wait = True)
    fold_no +=1

        

In [ ]:
def GRU_foldcv(params0):
    batch_size = params0['batch_size']
    epochs = params0['epochs']
    learning_rate = params0['learning_rate']
    activation = params0['activation']
    dropout_rate = params0['dropout_rate']
    optimizer = params0['optimizer']
    num_units = params0['num_units']
    num_layers = params0['num_layers']
    l1 = params0['lambda1']
    l2 = params0['lambda2']

    if optimizer == 'Adam':
        #opt = keras.optimizers.Adam(learning_rate=learning_rate)
        opt = tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate)
    elif optimizer == 'SGD':
         opt =optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=learning_rate)
         #opt = keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = tf.keras.optimizers.legacy.RMSprop(learning_rate=learning_rate)
        #opt = keras.optimizers.RMSprop(learning_rate=learning_rate)
  
    
    for i, (train_index, val_index) in enumerate(kfold.split(X_train, y_train)):       
            X_train_kfold, X_val_kfold = X_train[train_index], X_train[val_index]
            y_train_kfold, y_val_kfold = y_train[train_index], y_train[val_index]
            mchpt = tf.keras.callbacks.ModelCheckpoint(filepath=f'GRU/gru_new0_fold{i+1}_model{p}.ckpt',
                                                       monitor = 'val_loss' ,
                                                       save_best_only=True,
                                                       mode ='min', 
                                                       verbose =0)#,save_weights_only=True) restore_best_weights = True
            early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', 
                                                              patience=5, mode ='min'
                                                              ,verbose =0) # try patience of 3 or moin

            print(f'fitting{i+1}')
                        
            # Build the model
            Hybrid = Sequential()
            Hybrid.add(Input(shape=(X_train_kfold.shape[1],X_train_kfold.shape[2]), name='input_layer'))

            for i in range(num_layers):
                if i == num_layers - 1:
                    Hybrid.add(GRU(units=num_units,
                                   activation=activation,
                                   dropout=dropout_rate,
                                   return_sequences=False,
                                   name=f'Hidden_layer_{i+1}'))
                else:
                    # Not the last GRU layer
                    Hybrid.add(GRU(units=num_units,
                                   activation=activation,
                                   dropout=dropout_rate,
                                   return_sequences=True,
                                   name=f'Hidden_layer_{i+1}'))
            Hybrid.add(Dense(1))
            Hybrid.compile(loss=rmse, optimizer=opt, metrics=[r_squared])
            #Train the model on the entire training dataset
            Hybrid.fit(X_train_kfold, y_train_kfold,
                                   epochs = params0['batch_size'],
                                   batch_size = params0['batch_size'],
                                   shuffle = False,
                                   callbacks = [early_stopping,mchpt],
                                   validation_data=(X_val_kfold, y_val_kfold))
    return Hybrid

In [ ]:
def rmse(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 
def r_squared(y_true, y_pred):
    ss_res = K.sum(K.square(y_true - y_pred))
    ss_tot = K.sum(K.square(y_true - K.mean(y_true)))
    return 1 - ss_res / (ss_tot + K.epsilon())

In [ ]:
def GRU_foldcv(params0):
    batch_size = params0['batch_size']
    epochs = params0['epochs']
    learning_rate = params0['learning_rate']
    activation = params0['activation']
    dropout_rate = params0['dropout_rate']
    optimizer = params0['optimizer']
    num_units = params0['num_units']
    num_layers = params0['num_layers']
    l1 = params0['lambda1']
    l2 = params0['lambda2']

    if optimizer == 'Adam':
        #opt = keras.optimizers.Adam(learning_rate=learning_rate)
        opt = tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate)
    elif optimizer == 'SGD':
         opt =optimizer = tf.keras.optimizers.legacy.SGD(learning_rate=learning_rate)
         #opt = keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = tf.keras.optimizers.legacy.RMSprop(learning_rate=learning_rate)
        #opt = keras.optimizers.RMSprop(learning_rate=learning_rate)
  
    
    # for i, (train_index, val_index) in enumerate(kfold.split(X_train, y_train)):       
    #         X_train_kfold, X_val_kfold = X_train[train_index], X_train[val_index]
    #         y_train_kfold, y_val_kfold = y_train[train_index], y_train[val_index]
            # mchpt = tf.keras.callbacks.ModelCheckpoint(filepath=f'GRU/gru_new0_fold{i+1}_model{p}.ckpt',
            #                                            monitor = 'val_loss' ,
            #                                            save_best_only=True,
            #                                            mode ='min', 
            #                                            verbose =0)#,save_weights_only=True) restore_best_weights = True
            # early_stopping = tf.keras.callbacks.EarlyStopping(monitor = 'val_loss', 
            #                                                   patience=5, mode ='min'
            #                                                   ,verbose =0) # try patience of 3 or moin
                        
            # Build the model
    Hybrid = Sequential()
    Hybrid.add(Input(shape=(X_train_kfold.shape[1],X_train_kfold.shape[2]), name='input_layer'))

    for i in range(num_layers):
        if i == num_layers - 1:
            Hybrid.add(GRU(units=num_units,
                           activation=activation,
                           dropout=dropout_rate,
                           return_sequences=False,
                           name=f'Hidden_layer_{i+1}'))
        else:
            # Not the last GRU layer
            Hybrid.add(GRU(units=num_units,
                           activation=activation,
                           dropout=dropout_rate,
                           return_sequences=True,
                           name=f'Hidden_layer_{i+1}'))
    Hybrid.add(Dense(1))
    Hybrid.compile(loss=rmse, optimizer=opt, metrics=[rmse])
    #Train the model on the entire training dataset
    Hybrid.fit(X_train_kfold, y_train_kfold,
                           epochs = params0['batch_size'],
                           batch_size = params0['batch_size'],
                           shuffle = False,
                           # callbacks = [early_stopping,mchpt],
                           validation_data=(X_val_kfold, y_val_kfold))
    return Hybrid

In [ ]:
params0

In [ ]:
batch_size = 60
epochs = 20
learning_rate = 1e-10
activation = selu
dropout_rate = 0.2
opt = tf.keras.optimizers.legacy.Adam(learning_rate=learning_rate)
num_units = 20
l1 = 0.45
l2 = 0.26
# Build the model
Hybrid = Sequential()
Hybrid.add(Input(shape=(X_train_kfold.shape[1],X_train_kfold.shape[2]), name='input_layer'))

Hybrid.add(GRU(units=num_units,
               activation=activation,
               dropout=dropout_rate,
               return_sequences=False,
               name=f'Hidden_layer_{i+1}'))
Hybrid.add(Dense(1))
Hybrid.compile(loss=rmse, optimizer=opt, metrics=[rmse])
#Train the model on the entire training dataset
Hybrid.fit(X_train, y_train,
                       epochs = params0['batch_size'],
                       batch_size = params0['batch_size'],
                       shuffle = False,
                       # callbacks = [early_stopping,mchpt],
                       validation_data=(X_test, y_test))

In [ ]:
GRUparams = pd.read_csv('GRUparams1.csv', index_col = 'fold')
GRUparams

In [ ]:
for i in range(2):
    X_train_kfold,X_val_kfold = [X_train]*2
    y_train_kfold,y_val_kfold= [y_train]*2
    params0 =pd.read_csv('GRUparams1.csv', index_col = 'fold').iloc[i,:].to_dict()
    exec(f'G0{i+1} = GRU_foldcv(params0)')
    clear_output(wait = True)
    pred = eval(f'G0{i+1}.predict(X_test)')
    r2_score(y_test,pred)

In [ ]:
from skopt.callbacks import DeltaYStopper
r2_per_fold = []
loss_per_fold = []
rmse_per_fold = []
gru_best_score = []

kfold = KFold(n_splits=6)
fold_no = 1
parameters_fold = [None]*6
for i, (train_index, val_index) in enumerate(kfold.split(X_train, y_train)):
    print(i,fold_no)
    X_train_kfold, X_val_kfold = X_train[train_index], X_train[val_index]
    y_train_kfold, y_val_kfold = y_train[train_index], y_train[val_index]
    print(X_train_kfold.shape)
    # Best model for th k-fold
    params0 =pd.read_csv('GRUparams1.csv', index_col = 'fold').iloc[i,:].to_dict()
    exec(f'G{i+1} = GRU_foldcv(params0)')
    clear_output(wait = True)
    fold_no +=1

In [ ]:
weights_range = np.arange(0.05, 1.1, 0.18)
y_true = y_test
def concatenate_data(X_train, y_train, X_test, y_test):
    X = np.concatenate([X_train, X_test], axis=0)
    y = np.concatenate([y_train, y_test], axis=0)
    return X, y
def Unreshape_df(Xs, ys=None):
    X = np.concatenate(Xs, axis=0)
    if ys is not None:
        y = np.concatenate(ys, axis=0)
        return X, y
    else:
        return X

def rmse(y_true, y_pred):
        return np.sqrt(np.mean(np.square(y_pred - y_true)))  
def calculate_weighted_average(predictions, weights):
    return sum([weight*preds for preds, weight in zip(predictions, weights)])

def evaluate_ensemble(predictions, weights, y_true):
    pred = calculate_weighted_average(predictions, weights)
    loss = rmse(y_true,pred)
    return r2_score(y_true, pred),loss

#for i in range(2):
 #(f'GRU{i+1}')#
best_models = [G01,G02]
predictions = []
losses = []
for model in best_models:
    y_pred = model.predict(X_test)
    loss = rmse(y_true,y_pred)
    losses.append(loss)
    predictions.append(y_pred)
evaluate_ensemble(predictions, [0.6,0.4], y_true)

def Ensemble(X_topredict,best_weights,best_models):
    model_predictions = []
    for model in best_models:
        y_pred = model.predict(X_topredict).T[0]
        model_predictions.append(y_pred)

    # Combine the model predictions using the optimized weights to obtain the final prediction
    final_prediction = calculate_weighted_average(model_predictions, best_weights)
    return final_prediction
RNNe = Ensemble(X,[0.6,0.4],[G01,G02])

In [ ]:
v = scaler.inverse_transform(RNNe.reshape(-1, 1)).T[0]
RNNdf = pd.DataFrame().assign(RNNe = v,Time = plgm.iloc[100:-1,-3].values)
RNNdf.to_csv('RNNpred.csv')

In [ ]:
columns = [list(parameters_fold[0].keys())[0]] + list(parameters_fold[0][list(parameters_fold[0].keys())[1]].keys()) 
GRUparams = pd.DataFrame(columns = columns)
GRUparams.set_index('fold',inplace = True)
for ix in range(6):
    GRUparams.loc[ix] = ix+1
    for i in list(parameters_fold[ix][list(parameters_fold[0].keys())[1]].keys()):
        GRUparams.loc[ix,i] = parameters_fold[ix][list(parameters_fold[0])[1]][i]
GRUparams#.to_csv('GRUparams1.csv')

In [ ]:
import pickle
def rmse(y_true, y_pred):
        return K.sqrt(K.mean(K.square(y_pred - y_true))) 
    
r2 = []
loss = []
dic ={"r_squared": r_squared,"rmse":rmse}

for p in GRUparams.index:
    if p == 0 or p == 1:
        params0 = GRUparams.iloc[p,:].to_dict()
        G = GRU_foldcv(params0)
        clear_output(wait = True)
    

In [ ]:
#import load_model

r2 = []
loss = []
dic ={"r_squared": r_squared,"rmse":rmse}
best_models = []
for p in GRUparams.index:
    for i in list(range(0,7))[1:]:
        if p == 0 or p == 1:
            params = GRUparams.iloc[p,:].to_dict()
            exec(f'model{p}_{i} = load_model(f"GRU/gru_new0_fold{i}_model{p}.ckpt",dic)')
            exec(f'model{p}_{i}.compile(loss = rmse,\
                                     optimizer=keras.optimizers.Adam(learning_rate= params["learning_rate"]),\
                                    metrics = [r_squared])')
            print(f'fold{p}_{i}')
            exec(f'pred = model{p}_{i}.predict(X_test)')
            exec(f'best_models.append(model{p}_{i})')
            r = r2_score(pred,y_test);r2.append(r)
            loss.append(rmse(pred,y_test))
    clear_output()
for p in GRUparams.index:
    for i in list(range(0,7))[1:]:
        if p == 0 or p == 1:
            print(f'Fold:{i}, r2:{r2[i]}, rmse: {loss[i]}')

In [ ]:
def get_input_gradients(model, x):
    # Convert the input to a tensor
    x = np.reshape(x, (x.shape[0], 100, 9))
    x_tensor = tf.convert_to_tensor(x)
    
    # Compute the gradients of the output with respect to the input
    with tf.GradientTape() as tape:
        tape.watch(x_tensor)
        output = model(x_tensor)
        
    gradients = tape.gradient(output, x_tensor)
    
    # Convert the gradients to a numpy array
    gradients = tf.keras.backend.get_value(gradients)
    
    # Compute the absolute sum of the gradients for each feature
    feature_importances = np.sum(np.abs(gradients), axis=0)
    
    return feature_importances

# Compute the feature importances for a sample input
x_sample = X_train[0:1, :, :]
feature_importances = get_input_gradients(G1, x_sample)

# Print the feature importances
#print("Feature importances:", feature_importances)

# Compute the average feature importances across all timesteps
average_importances = np.mean(feature_importances, axis=0)

# Rank the features based on their importance
sorted_indices = np.argsort(average_importances)[::-1]

# Print the feature rankings
names = list(df.columns)[:-1]
for i in range(len(sorted_indices)):
    print(f"{i+1}. {names[sorted_indices[i]]}: {average_importances[sorted_indices[i]]}")


In [ ]:
def get_input_gradients(model, x):
    # Convert the input to a tensor
    x = np.reshape(x, (x.shape[0], 100, 9))
    x_tensor = tf.convert_to_tensor(x)
    
    # Compute the gradients of the output with respect to the input
    with tf.GradientTape() as tape:
        tape.watch(x_tensor)
        output = model(x_tensor)
        
    gradients = tape.gradient(output, x_tensor)
    
    # Convert the gradients to a numpy array
    gradients = tf.keras.backend.get_value(gradients)
    
    # Compute the absolute sum of the gradients for each feature
    feature_importances = np.sum(np.abs(gradients), axis=0)
    
    return feature_importances

# Compute the feature importances for a sample input
x_sample = X_train[0:1, :, :]
feature_importances = get_input_gradients(G2, x_sample)

# Print the feature importances
#print("Feature importances:", feature_importances)

# Compute the average feature importances across all timesteps
average_importances = np.mean(feature_importances, axis=0)

# Rank the features based on their importance
sorted_indices = np.argsort(average_importances)[::-1]

# Print the feature rankings
names = list(df.columns)[:-1]
for i in range(len(sorted_indices)):
    print(f"{i+1}. {names[sorted_indices[i]]}: {average_importances[sorted_indices[i]]}")


In [ ]:
weights_range = np.arange(0.05, 1.1, 0.18)
y_true = y_test
def concatenate_data(X_train, y_train, X_test, y_test):
    X = np.concatenate([X_train, X_test], axis=0)
    y = np.concatenate([y_train, y_test], axis=0)
    return X, y
def Unreshape_df(Xs, ys=None):
    X = np.concatenate(Xs, axis=0)
    if ys is not None:
        y = np.concatenate(ys, axis=0)
        return X, y
    else:
        return X

def rmse(y_true, y_pred):
        return np.sqrt(np.mean(np.square(y_pred - y_true)))  
def calculate_weighted_average(predictions, weights):
    return sum([weight*preds for preds, weight in zip(predictions, weights)])

def evaluate_ensemble(predictions, weights, y_true):
    pred = calculate_weighted_average(predictions, weights)
    loss = rmse(y_true,pred)
    return r2_score(y_true, pred),loss

#for i in range(2):
 #(f'GRU{i+1}')#
best_models = best_modelss
predictions = []
losses = []
for model in best_models:
    y_pred = model.predict(X_test)
    loss = rmse(y_true,y_pred)
    losses.append(loss)
    predictions.append(y_pred)
iterations = len(weights_range) ** len(best_models)
best_weights = None
best_score = None
best_loss = None
track= {}
si = 0
bi = 0
found = lambda si,bi,best_score: si> 0 and bi> 0 and si > bi*10000#or (best_score is not None and best_score > 0.91)
for wx0, w1 in enumerate(weights_range):
    if found(si,bi,best_score):
        wx0 = weights_range[len(weights_range)-1]
        break
    for wx1, w2 in enumerate(weights_range):
        if found(si,bi,best_score):
            wx1 = weights_range[len(weights_range)-1]
            break
        for wx2, w3 in enumerate(weights_range):
            if found(si,bi,best_score):
                wx2 = weights_range[len(weights_range)-1]
                break
            for wx3, w4 in enumerate(weights_range):
                if found(si,bi,best_score):
                    wx3 = weights_range[len(weights_range)-1]
                    break
                for wx5, w6 in enumerate(weights_range):
                    if found(si,bi,best_score):
                        wx5 = weights_range[len(weights_range)-1]
                        break
                    for wx6, w7 in enumerate(weights_range):
                        if found(si,bi,best_score):
                            wx6 = weights_range[len(weights_range)-1]
                            break
                        for wx7, w8 in enumerate(weights_range):
                            if found(si,bi,best_score):
                                wx7 = weights_range[len(weights_range)-1]
                                break
                            for wx8, w9 in enumerate(weights_range):
                                if found(si,bi,best_score):
                                    wx8 = weights_range[len(weights_range)-1]
                                    break
                                for wx9, w10 in enumerate(weights_range):
                                    if found(si,bi,best_score):
                                        wx9 = weights_range[len(weights_range)-1]
                                        break
                                    for wx10, w11 in enumerate(weights_range):
                                        if found(si,bi,best_score):
                                            wx10= weights_range[len(weights_range)-1]
                                            break
                                        for wx12, w13 in enumerate(weights_range):
                                            if found(si,bi,best_score):
                                                wx12 = weights_range[len(weights_range)-1]
                                                break
                                            if found(si,bi,best_score):
                                                wx6 = weights_range[len(weights_range)-1]
                                                clear_output(wait = True)
                                                print(f'{best_score:.2%}')
                                                print(f'{found(si,bi,best_score)}')
                                                print(f'{si},{bi}')
                                                break
                                            else:
                                                w = [random.choice(weights_range) for _ in range(12)]
                                                weights = [nu/sum(w) for nu in w]
                                                score = evaluate_ensemble(predictions, weights, y_true)[0]
                                                s_loss = evaluate_ensemble(predictions, weights, y_true)[1]
                                                si += 1
                                                cond = (best_score is None or best_loss is None) or\
                                                       (np.round(score,4) > np.round(best_score,4) and np.round(s_loss, 2) < 0.12)
                                                if cond:
                                                    bi += 1
                                                    best_score = score       
                                                    best_weights = weights
                                                    best_loss = s_loss
                                                    track[bi] = {'Titer':si,'score':best_score , 'loss':best_loss}
                                                    clear_output(wait = True)
                                                    print(f'{best_loss}')
                                                    print(f'{best_score:.2%}')
                                                    print(f'{best_weights}')
                                                    print(f'{found(si,bi,best_score)}')
                                                    print(f'recent')

                                                if si> 0 and bi> 0:
                                                    clear_output(wait = True)
                                                    print(f'{score:.2%}')
                                                    print(f'{loss}')
                                                    print(f'{weights}')
                                                    print(f'{found(si,bi,best_score)}')
                                                    print(f'{si},{bi}')
                                                    print(f'#### Best ####')
                                                    print(f'{best_score:.2%}')
                                                    print(f'{best_loss}')
                                                    print(f'{best_weights}')


                                            if found(si,bi,best_score):
                                                break
                                        if found(si,bi,best_score):
                                            break
                                    if found(si,bi,best_score):
                                        break
                                if found(si,bi,best_score):
                                    break
                            if found(si,bi,best_score):
                                break
                        if found(si,bi,best_score):
                            break
                    if found(si,bi,best_score):
                        break
                if found(si,bi,best_score):
                    break
            if found(si,bi,best_score):
                break
        if found(si,bi,best_score):        
            track[bi] = {'Titer':si,'score':best_score , 'loss':best_loss}
            print(best_weights, best_score)
            break
                #if i == 0:
                #    best_weights1 = best_weights
                #else:
                #    best_weights2 = best_weights
                #best_w = [best_weights1,best_weights2]         


In [ ]:
def Ensemble(X_topredict,best_weights,best_models):
    model_predictions = []
    for model in best_models:
        y_pred = model.predict(X_topredict).T[0]
        model_predictions.append(y_pred)

    # Combine the model predictions using the optimized weights to obtain the final prediction
    final_prediction = calculate_weighted_average(model_predictions, best_weights)
    return final_prediction

pred = list(Ensemble(X_test,best_weights,best_modelss))
clear_output()
print(r2_score(y_true, pred),rmse(y_true,pred))
#for i in range(2):
#    print(f'GRU{i+1}')
#    best_models = best_modelss[i]
#    best_weights = best_w[i]
#    exec(f'Ens_pred{i} = list(Ensemble(X_test,best_weights,best_models))')
#    print(r2_score(y_true, eval(f'Ens_pred{i}')),rmse(y_true,eval(f'Ens_pred{i}')))
    

In [ ]:
def get_input_gradients(model, x):
    # Convert the input to a tensor
    x = np.reshape(x, (x.shape[0], 100, 9))
    x_tensor = tf.convert_to_tensor(x)
    
    # Compute the gradients of the output with respect to the input
    with tf.GradientTape() as tape:
        tape.watch(x_tensor)
        output = model(x_tensor)
        
    gradients = tape.gradient(output, x_tensor)
    
    # Convert the gradients to a numpy array
    gradients = tf.keras.backend.get_value(gradients)
    
    # Compute the absolute sum of the gradients for each feature
    feature_importances = np.sum(np.abs(gradients), axis=0)
    
    return feature_importances

# Compute the feature importances for a sample input
x_sample = X_train[0:1, :, :]
feature_importances = get_input_gradients(model0_1, x_sample)

# Print the feature importances
#print("Feature importances:", feature_importances)

# Compute the average feature importances across all timesteps
average_importances = np.mean(feature_importances, axis=0)

# Rank the features based on their importance
sorted_indices = np.argsort(average_importances)[::-1]

# Print the feature rankings
names = list(df.columns)[:-1]
for i in range(len(sorted_indices)):
    print(f"{i+1}. {names[sorted_indices[i]]}: {average_importances[sorted_indices[i]]}")


In [ ]:
def compute_feature_importances(models, weights, X, y):
    """
    Computes the feature importances using the representation erasure approach for an ensemble of models.
    
    Args:
        models (list): A list of TensorFlow or Keras models.
        weights (list): A list of floating point numbers representing the weights of each model in the ensemble.
        X (numpy.ndarray): An array of shape (n_samples, n_features) containing the input data.
        y (numpy.ndarray): An array of shape (n_samples,) containing the target labels.
    
    Returns:
        A numpy.ndarray of shape (n_features,) containing the feature importances.
    """
    baseline_score = 0
    for i, model in enumerate(models):
        pred = model.predict(X)
        score = rmse(y,pred)
        baseline_score += score * weights[i]
    baseline_score /= np.sum(weights)

    feature_importances = np.zeros(X.shape[1])
    clear_output()
    for i in range(X.shape[1]):
        X_test = np.copy(X)
        X_test[:, i] = 0
        score = 0
        for j, model in enumerate(models):
            pred = model.predict(X)
            score +=  rmse(y,pred) * weights[j]
        feature_importances[i] = np.abs(baseline_score - score)

    # Normalize the feature importances
    feature_importances = feature_importances / np.sum(feature_importances)
    
    return feature_importances
best_weights = [0.015151515151515152, 0.06969696969696969, 0.06969696969696969, 0.015151515151515152, 
                0.015151515151515152, 0.015151515151515152, 0.015151515151515152, 0.015151515151515152, 
                0.2333333333333333, 0.1787878787878788, 0.2333333333333333, 0.12424242424242422]
fi = compute_feature_importances(best_models, best_weights, X_train, y_train)


In [ ]:
best_weights = [0.6,0.4]

In [ ]:
feature_importances = np.zeros(X.shape[2])
clear_output()
best_weights = [0.6,0.4]
best_models = [G01,G02]

baseline_score = 0
for i, model in enumerate(best_models):
    pred = model.predict(X)
    score = np.array(rmse(y,pred))
    baseline_score += score * best_weights[i]
baseline_score /= np.sum(best_weights)
for i in range(X.shape[2]):
    X_test = np.copy(X)
    X_test[:,:,i] = 0
    score = 0
    for j, model in enumerate(best_models):
        pred = model.predict(X_test)
        score +=  rmse(y,pred) * best_weights[j]
    feature_importances[i] = np.abs(baseline_score - score)

In [ ]:
sorted_indices = np.argsort(feature_importances)[::-1]
names = list(df.columns)[:-1]
nm = {}
for i in range(len(sorted_indices)):
    nm[names[sorted_indices[i]]] = {'value':feature_importances[sorted_indices[i]],
                                    'N°':int(i+1)}
    print(f"{i+1}. {names[sorted_indices[i]]} : {feature_importances[sorted_indices[i]]}")

In [ ]:
FI = pd.DataFrame(nm).T
FI['value'] = np.round(FI['value'] / np.sum(FI['value']),3)
[print(x) for x in FI.value]

In [ ]:
dic_importance = {}
for ix, model in enumerate(best_models):
    feature_importances = get_input_gradients(model, x_sample)
    average_importances = np.mean(feature_importances, axis=0)
    sorted_indices = np.argsort(average_importances)[::-1]
    feature_rankings = {names[sorted_indices[i]]: average_importances[sorted_indices[i]] for i in range(len(sorted_indices))}
    dic_importance[ix] = {'importance':feature_rankings}



In [ ]:
result = {}
for _, sub_dict in dic_importance.items():
    for key, value in subdict['importance'].items():
        if key not in result:
            result[key] = []
        result[key].append(value)

for key in result:
    result[key] =sum(w * v for w, v in zip(best_weights, result[key])) / sum(best_weights)
result = dict(sorted(result.items(), key=lambda item: item[1],reverse=True))
result
    

In [ ]:
pred = scaler.inverse_transform(Ensemble(X_test,best_weights,best_modelss).reshape(-1, 1)).T

In [ ]:
import pickle

In [ ]:
pred = scaler.inverse_transform(Ensemble(X,best_weights,best_models).reshape(-1, 1)).T
with open('RNNpred.pkl', 'wb') as f:
    pickle.dump(pred[0], f)

In [ ]:
p =Ensemble(X,best_weights,best_models)

In [ ]:
p.shape

In [ ]:
predictions = [Ens_pred0]
predictions.append(Ens_pred1)
#losses = [rmse(y_true,Ens_pred0),rmse(y_true,Ens_pred1)]
iterations = len(weights_range) ** 2
best_weights = None
best_score = None
best_loss = None
track= {}
si = 0
bi = 0
found = lambda si,bi,best_score: si> 0 and bi> 0 and si > bi*10000 or best_score is not None and best_score > 0.91
    
for wx0, w1 in enumerate(weights_range):    
    if found(si,bi,best_score):
        wx0 = weights_range[len(weights_range)-1]
        break
    for wx1, w2 in enumerate(weights_range):
        if found(si,bi,best_score):
            wx1 = weights_range[len(weights_range)-1]
            clear_output(wait = True)
            print(f'{best_score:.2%}')
            print(f'{found(si,bi,best_score)}')
            print(f'{si},{bi}')
            break
        else:
            w = [random.choice(weights_range) for _ in range(2)]
            weights = [nu/sum(w) for nu in w]
            score = evaluate_ensemble(predictions, weights, y_true)[0]
            s_loss = evaluate_ensemble(predictions, weights, y_true)[1]
            si += 1
            cond = (best_score is None or best_loss is None) or (np.round(score,4) > np.round(best_score,4) and np.round(s_loss, 2) < 0.12)
            if cond:
                bi += 1
                best_score = score       
                best_weights = weights
                if best_loss is None:
                    best_loss = s_loss
                track[bi] = {'Titer':si,'score':best_score , 'loss':best_loss}
                if (np.round(s_loss,2) < np.round(best_loss, 2)):
                    if not cond:
                        bi += 1
                    best_score = score 
                    best_loss = s_loss
                    best_weights = weights        
                    track[bi] = {'Titer':si,'score':best_score , 'loss':best_loss}
                clear_output(wait = True)
                print(f'{best_loss}')
                print(f'{best_score:.2%}')
                print(f'{best_weights}')
                print(f'{found(si,bi,best_score)}')
                print(f'recent')

            if si> 0 and bi> 0:
                clear_output(wait = True)
                print(f'{score:.2%}')
                print(f'{loss}')
                print(f'{weights}')
                print(f'{found(si,bi,best_score)}')
                print(f'{si},{bi}')
                print(f'#### Best ####')
                print(f'{best_score:.2%}')
                print(f'{best_loss}')
                print(f'{best_weights}')

            if found(si,bi,best_score):
                wx1 = weights_range[len(weights_range)-1]
                break
    if found(si,bi,best_score):
        track[bi] = {'Titer':si,'score':best_score , 'loss':best_loss}
        print(best_weights, best_score)


In [ ]:
r2 = []
loss = []
for i in list(range(0,7))[1:]:
    exec(f'model{i} = load_model("GRU/gru_{i}_fold.ckpt",dic)')
    print(f'fold{i}')
    exec(f'pred{i} = model{i}.predict(X_test)')
    r = eval(f'r2_score(pred{i},y_test)');r2.append(r)
    eval(f'loss.append(rmse(pred{i},y_test))')
clear_output()
for i in list(range(0,6)):
    print(i)
    print(f'Fold:{i+1}, r2:{r2[i]}, rmse: {loss[i]}')

In [ ]:
dic = {'r_squared':r_squared,'rmse':rmse}

r2 = []
loss = []
for i in list(range(0,7))[1:]:
    exec(f'model{i} = load_model("GRU/gru_{i}_fold.ckpt",dic)')
    print(f'fold{i}')
    exec(f'pred{i} = model{i}.predict(X_test)')
    r = eval(f'r2_score(pred{i},y_test)');r2.append(r)
    eval(f'loss.append(rmse(pred{i},y_test))')
clear_output()
for i in list(range(0,6)):
    print(i)
    print(f'Fold:{i+1}, r2:{r2[i]}, rmse: {loss[i]}')

In [ ]:
#r2_score(y,Ens_pred),rmse(y,Ens_pred)
r2_score(y_true,Ens_pred),rmse(y_true,Ens_pred)

### RNN

In [ ]:
from skopt.callbacks import DeltaYStopper
n_splits = 6
kfold = KFold(n_splits=n_splits, shuffle=False)

# Define the search space
search_space = [Categorical([10,20,40], name='batch_size'),
                Categorical([10], name='epochs'),
                Real(1e-6, 1e-3 ,'log-uniform',name='learning_rate'),
                Categorical(['relu', 'tanh','selu','elu'], name='activation'),
                Real(0.0, 0.5, name='dropout_rate'),
                Categorical(['Adam', 'SGD','rmsprop'], name='optimizer'),
                Integer(50, 200, name='num_units'),
                Integer(1, 3, name='num_layers'),
                Real(0.1,0.9,name='lambda1'), 
                Real(0.1,0.9,name='lambda2')]

@use_named_args(search_space)
def objective(**params):
    # Unpack the hyperparameters
    batch_size = params['batch_size']
    epochs = params['epochs']
    learning_rate = params['learning_rate']
    activation = params['activation']
    dropout_rate = params['dropout_rate']
    optimizer = params['optimizer']
    num_units = params['num_units']
    num_layers = params['num_layers']
    
    if optimizer == 'Adam':
        opt = keras.optimizers.Adam(learning_rate=learning_rate)
    elif optimizer == 'SGD':
        opt = keras.optimizers.SGD(learning_rate=learning_rate)
    elif optimizer == 'rmsprop':
        opt = keras.optimizers.RMSprop(learning_rate=learning_rate)

    
    GRUmodel = Sequential()
    GRUmodel.add(Input(shape=(X_train.shape[1],X_train.shape[2]), name='input_layer'))
    for i in range(num_layers):
        if i == num_layers - 1:
            GRUmodel.add(GRU(units=num_units,
                           activation=activation,
                           dropout=dropout_rate,
                           return_sequences=False,
                           name=f'Hidden_layer_{i+1}'))
        else:
            # Not the last GRU layer
            GRUmodel.add(GRU(units=num_units,
                           activation=activation,
                           dropout=dropout_rate,
                           return_sequences=True,
                           name=f'Hidden_layer_{i+1}'))
    GRUmodel.add(Dense(1))
    GRUmodel.compile(loss=rmse, optimizer= opt, metrics=[r_squared])
    
    print(f)
    # Train the model on the training data for this fold
    history = GRUmodel.fit(X_train_kfold, y_train_kfold,
                           batch_size=batch_size,
                           epochs=epochs)
    
    # Evaluate the model on the validation data for this fold
    val_loss, val_r_squared = GRUmodel.evaluate(X_val_kfold, y_val_kfold)
    
    return val_loss

delta_stopper = DeltaYStopper(delta=0.1)
for train_index, val_index in kfold.split(X_train, y_train):
    X_train_kfold, X_val_kfold = X_train[train_index], X_train[val_index]
    y_train_kfold, y_val_kfold = y_train[train_index], y_train[val_index]
    
    # Perform Bayesian optimization on the training data for this fold
    res_gp = gp_minimize(objective,
                         search_space,
                         n_calls=15,
                         callback=[delta_stopper],
                         random_state=42)
    
    # Print the best score and best parameters for this fold
    print(f"Best score for fold {i}: {res_gp.fun} using {res_gp.x}")

In [ ]:
pred = model.predict(X_test)
r2_score(y_test,pred),rmse(y_test,pred)

In [ ]:
def Unreshape_df(Xs, ys=None):
    X = np.concatenate(Xs, axis=0)
    if ys is not None:
        y = np.concatenate(ys, axis=0)
        return X, y
    else:
        return X
    


In [ ]:
scaler.inverse_transform(unpred.reshape(-1, 1)).T

In [ ]:
unpred = Unreshape_df(Xs = pred)
predicted = scaler.inverse_transform(unpred.reshape(-1, 1)).T[0]
observed =  scaler.inverse_transform(Unreshape_df(Xs = y_test).reshape(-1, 1)).T[0]
r2_score(predicted,observed)

In [ ]:
from sklearn.model_selection import KFold
from bayes_opt import BayesianOptimization

optimizers = {'Adam':Adam, 'SGD':SGD, 'RMSprop':RMSprop, 'Adamax':Adamax, 'Nadam':Nadam}
activation_functions = ['selu','elu',LeakyReLU,'relu'] #()

# Define the hyperparameter search space:
pbounds = {'dropout_rate': (0.1, 0.5),
           'num_neurons': (8,256),
           'learning_rate': (-7, -2),
           'activation_function': (0, len(activation_functions)-1),
           'optimizer': (0, len(optimizers)-1),
           'num_hidden_layers': (1, 3),
           'epochs': (5,100),
           'batch_size': (8, 128)
          }

# Define a function to train and evaluate the RNN model for a given set of hyperparameters:
def rnn_evaluate(epochs,batch_size,dropout_rate, num_neurons, learning_rate, activation_function, optimizer, num_hidden_layers):
    
    # Convert integer indices back to hyperparameters:
    activation_function = activation_functions[int(activation_function)]
    optimizer = list(optimizers.values())[int(optimizer)](learning_rate=learning_rate)
    
    # Perform 6-fold cross validation:
    num_folds = 6
    kf = KFold(n_splits=num_folds, shuffle=True)
    acc_list = []
    es = EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)
    for train_idx, test_idx in kf.split(X_train,y_train):
        X_Train, X_val = X[train_idx], X[test_idx]
        y_Train, y_val = y[train_idx], y[test_idx]
        
        # Define and train the RNN model:
        model = Sequential()
        model.add(SimpleRNN(units=int(num_neurons), input_shape=(X_train.shape[1], X_train.shape[2])))
        for i in range(int(num_hidden_layers)-1):
            model.add(Dense(num_neurons, activation=activation_function))
            model.add(Dropout(dropout_rate))
        model.add(Dense(1, activation='sigmoid'))
        model.compile(loss=rmse, optimizer=optimizer, metrics=[r_squared])
        #model.fit(X_Train, y_Train, epochs=10, batch_size=32,validation_data=(X_val, y_val), verbose=0)
        #model.fit(X_Train, y_Train, epochs=int(epochs), batch_size= int(batch_size),validation_data=(X_val, y_val),verbose=0)
        history = model.fit(X_Train, y_Train, epochs=int(epochs), batch_size=int(batch_size),
                    validation_data=(X_val, y_val),
                    verbose=0, callbacks=[es])   
        
        y_pred = scaler.inverse_transform(model.predict(X_test))
        mse_v = mean_squared_error(scaler.inverse_transform(y_test), y_pred)
        rmse_v = np.sqrt(mse_v)
        acc_list.append(rmse_v)
    
    # Return the mean accuracy over all folds as the performance metric of interest:
    return sum(acc_list) / len(acc_list)

# Perform Bayesian optimization to find the optimal hyperparameters:
optimizer = BayesianOptimization(f=rnn_evaluate, pbounds=pbounds, random_state=42)
optimizer.maximize(init_points=5, n_iter=50)


In [ ]:
print(optimizer.max)  

In [ ]:
#from tensorflow.keras.models import Sequential
#from tensorflow.keras.layers import Dense, LSTM
#from tensorflow.keras.optimizers import Adam

# Define the optimal hyperparameters found by the optimizer

def RNN_PLGM(X,param_dict,s = 0.8309):
    # Convert integer indices back to hyperparameters:
    optimizers = {'Adam':Adam, 'SGD':SGD, 'RMSprop':RMSprop, 'Adamax':Adamax, 'Nadam':Nadam}
    activation_functions = ['selu','elu',LeakyReLU,'relu'] 
    
    my_dict= param_dict    
    batch_size = int(my_dict['batch_size'])
    dropout_rate = my_dict['dropout_rate']
    epochs = int(my_dict['epochs'])
    learning_rate = my_dict['learning_rate']
    num_hidden_layers = my_dict['num_hidden_layers']
    num_neurons = my_dict['num_neurons']
    activation_function = activation_functions[int(my_dict['activation_function'])]
    optimizer = list(optimizers.values())[int(my_dict['optimizer'])](learning_rate=learning_rate)
    
    split = int((s)*X.shape[0])
    X_train = X[:split,:,:]
    y_train = y[:split,:]
    X_test =  X[split:,:,:]
    y_test =  y[split:,:]
    
    # Define and train the RNN model:
    RNN = Sequential()
    RNN.add(SimpleRNN(units=int(num_neurons), input_shape=(X_train.shape[1], X_train.shape[2])))
    for i in range(int(num_hidden_layers)-1):
        RNN.add(Dense(num_neurons, activation=activation_function))
        RNN.add(Dropout(dropout_rate))
    RNN.add(Dense(1, activation='sigmoid'))
    RNN.compile(loss=rmse, optimizer=optimizer, metrics=[r_squared])
    # Train the model with your data
    RNN.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.2)

    # Make predictions on new data
    y_pred = RNN.predict(X_test)
    
    return y_pred


In [ ]:
# Define the optimal hyperparameters obtained from optimization
# Define and compile the RNN model with optimal hyperparameters
param_dict = my_dict 
batch_size = int(my_dict['batch_size'])
dropout_rate = my_dict['dropout_rate']
epochs = int(my_dict['epochs'])
learning_rate = my_dict['learning_rate']
num_hidden_layers = my_dict['num_hidden_layers']
num_neurons = my_dict['num_neurons']
activation_function = activation_functions[int(my_dict['activation_function'])]
optimizer = list(optimizers.values())[int(my_dict['optimizer'])](learning_rate=learning_rate)

model = Sequential()
model.add(SimpleRNN(units=int(num_neurons), input_shape=(X_train.shape[1], X_train.shape[2])))
for i in range(int(num_hidden_layers)-1):
    model.add(Dense(num_neurons, activation=activation_function))
    model.add(Dropout(dropout_rate))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss=rmse, optimizer=optimizer, metrics=[r_squared])

# Train the model on the entire training set with optimal hyperparameters
history = model.fit(X_train, y_train, epochs=int(epochs), batch_size=int(batch_size),
                    verbose=1)

# Evaluate the model on the test set
y_pred = scaler.inverse_transform(model.predict(X_test))
mse_v = mean_squared_error(scaler.inverse_transform((y_test), y_pred)
rmse_v = np.sqrt(mse_v)
r2 = r2_score(scaler.inverse_transform((y_test), y_pred)
print('RMSE:', rmse_v,'R2',r2 )


In [ ]:
#y_pred = scaler.inverse_transform(model.predict(X_test))
#X_train: (13006, 100, 15)
#X_test: (2200, 100, 15)
#y_train: (13006, 1)
#y_test: (2200, 1)
p =y_test.reshape(2200, 1)
scaler = MinMaxScaler(feature_range=(0, 1))
scaler.inverse_transform(p)